In [2]:
# Learning OOP through A Library System


from abc import ABC, abstractmethod

# ENCAPSULATION - Keep things together that belong together

# GOAL
# In procedural programming, you might have seperate variables for book title, and borrowed status,
# and multiple functions manipulating them
# Encapsulation fixes that by grouping "data" and "behavior" inside a single class - the Book
# This creates a clear boundary: the only way to change the book's state is through its own method


class Book:
  """ Represents a single book in the library."""

  def __init__(self, title, author):
    # These are instance attributes
    # They define what data each Book object "owns"
    # They are like the book's DNA - unique to every instance
    self.title = title
    self.autor = author
    self._is_borrowed = False # underscore = "protected" variable (internal use)

  def borrow(self):
    """
    This method allows borrowing the book.
    It "controls access" to the internal '_is_borrowed' variable


    Why this helps Encapsulation:
    - It prevents external code from directly modifying '_is_borrowed'.
    - It ensures that only valid operations happen (you can't borrow twice).


    Without this method:
    - someone could directly do '_is_borrowed = True' even if the book was already borrowed
    - You would lose control and consistency in your system.

    """

    if not self._is_borrowed:
      self._is_borrowed = True
      print(f"You borrowed '{self.title}'.")
    else:
      print(f"{self.title}' is already borrowed.")

  def return_book(self):
    """
    This reverses the borroowing action - another controlled entry point.

    Why it exists:
    - Maintains the integrity of state changes.
    - Prevents misuse, like returning an unborrowed book.

    """

    if self._is_borrowed:
      self._is_borrowed = False
      print(f"You returned '{self.title}'.")
    else:
      print((f"{self.title}' was not borrowed."))

  def display_info(self):
    """
    Display the book's current details.
    Demonstrates that the internal state can be "safely read" withoutexposing it directly.

    """

    status = "Available" if not self._is_borrowed else "Borrowed"
    print(f"{self.title}' by {self.author} - {status}")


# The Member class also shows Encapsulation in action.
# A member's list of borrowed books is internal (not exposed)
# You interact with it through public methods (borrow_book, return_book, show_borrowed_books)

class Member:
  """ Represents a library member."""


  def __init__(self, name):
    # Each member has their own private list of borrowed books.
    self.name = name
    self._borrowed_books = [] # Internal data (shouldn't be touched directly)


  def borrow_book(self, book: Book):
    """
    This method coordinates the borrowing process.
    It calls the Book's own borrow() method - repecting that the Books controls its own state


    Why this matters:
    - The Member does not directly set the Book's variables.
    - It only requests the Book to perform its own logic.
    This preserves Encapsulation between classes too.
    """

    if not book._is_borrowed:
      book.borrow()
      self._borrowed_books.append(book)
    else:
      print(f"{self.name} cannot borrow '{book.title}' - already taken")

  def return_book(self, book: Book):
    """
    This ensures a member can only return books they actually borrowed.
    Prevents external tampering and enforces logical consistency.
    """

    if book in self._borrowed_books:
      book.return_book()
      self._borrowed_books.remove(book)
    else:
      print(f"{self.name} didn't borrow '{book.title}'.")

  def show_borrowed_books(self):
    """
    Displays which books this member has borrowed.
    Still doesn't expose the internal list directly - only reports its contents.
    """

    print(f"\nBooks borrowed by {self.name}:")
    if not self._borrowed_books:
      print(" (none)")
    else:
      for b in self._borrowed_books:
        print(f"  -{b.title}")


# Without Encpsulation
# - Anyone could cahnge book._is_borrowed = True directly
# - Member could tamper with another Member's books.
# - No control, no rules, chaos




# INHERITANCE - "Reuse what's common, specialize what's different"


# Goal:
# To avoid  rewriting identical logic (like borrow() and return_item()) from every type of item.
# Instead, define shared bahavior in a base class (LibraryItem),
# then extend it with specialized subclasses.


class LibraryItem(ABC):
  """
  This is the "base class" (parent) for all items in the library.
  It captures the shared structure and bahavior among Books, Magazines, etc.

  """


  def __init__(self, title):
    self.title = title
    self._is_borrowed = False


  def borrow(self):
    """Same logic for all items."""
    if not self._is_borrowed:
      self._is_borrowed = True
      print(f"You borrowed '{self.title}'.")
    else:
      print(f"{self.title}' is already borrowed.")


  def return_item(self):
    """Same logic for all items."""
    if self._is_borrowed:
      self._is_borrowed = False
      print(f"You returned '{self.title}'.")
    else:
      print(f"{self.title}' wasn't borrowed.")


  @abstractmethod
  def display_info(self):
    """
    Abstract method- child classes MUST implement this.
    This enforces that every type of LibraryItem knows how to display itself.
    (This part also demonstrates Abstraction later.)
    """

    pass


# Subclasses specialize behavior.
class BookItem(LibraryItem):
  """
  A specific type of LibraryItem- adds an author attribute.
  Inherits borrowing and returning logic LibraryItem.
  """

  def __init__(self, title, author):
    super().__init__(title)   # Calls parent's contructor to initialize shared data
    self.author = author


  def display_info(self):
    """ Specialized version of display_info for books."""
    status = "Available" if not self._is_borrowed else "Borrowed"
    print(f"[Book] '{self.title}' by {self.author} - {status}")


class MagazineItem(LibraryItem):
  """
  Another subclass - same parent, different extra attribute (issue_number).
  """

  def __init__(self, title, issue_number):
    super().__init__(title)
    self.issue_number = issue_number

  def display_info(self):
    """Magazine's own way of showing details."""
    status = "Available" if not self._is_borrowed else "Borrowed"
    print(f"[Magazine] '{self.title}' (Issue{self.issue_number}) - {status}")


# Without Inheritance
# We would have to write seperate borrow() and return() for every item type
# Changes (like new borrow policy) would require editing multiple classes.
# The code would balloon in size and bugs would multiply.




# POLYMORPHISM - Different types, same interface

# Goal:
# Let different object types respond to the same "function call" in their way.
# Here, both BookItem and MagazineItem share a method name: display_info().
# When we loop through items, Python automatically calls the correct one.


def show_library_items(items):
  """
  This function demonstrates polymorphism.
  It calls the same method name (display_info) on each item,
  but the output changes depending on the object's class.


  Why this matters:
  - You don't need 'if type(item) == ...' checks
  - You can add new item types later (e.g, DVDItem) without changing this function.

  """

  print("\nLibrary Catalog:")
  for item in items:
    item.display_info()   # Python dynamically resolves the right version!


# Without Polymorphism
# You would have ugly chains like:
# if isinstance(item, BookItem):....
# elif isinstances(item, MagazineItem)....
# That defeats the purpose of OOP - flexibility





# ABSTRACTION: Show only what is necessary; hide the rest


# GOAL
# Abstraction hides how something works and only exposes what it does.
# The user dosn't need to know how borrow() modifies variables -only it does



# In this program:
# The base class (LibraryItem) defines a general template.
# Subclasses fill in the details
# The abstract method 'display_info()' enforces a structure while hiding complexity.


# Without Abstraction
# - Every part of the code would need to know internal implementation details
# - Changing internal logic (like remaining '_is_borrowed') would break everything




# PUTTING EVERYTHING TOGETHER



if __name__ == "__main__":
  # Create specific items
  book1 = BookItem("Python Crash Course", "Eric Matthes")
  book2 = BookItem("Clean Code", "Robert C. Martin")
  mag1 = MagazineItem("National Geographic", "Nov 2025")


  # Combine them into catalog
  library_items = [book1, book2, mag1]


  # Create a library member
  member = Member("Nikhil")


  # Display catalog - Polymorphism in action
  show_library_items(library_items)



  # Borrow and return actions - Encapsulation in action
  member.borrow_book(book1)
  member.borrow_book(mag1)
  member.show_borrowed_books()


  member.borrow_book(book1)   # borrowing again -> prevented by Encapsulation
  member.return_book(book1)
  member.show_borrowed_books()


  # Final catalog - shows updated availability

  show_library_items(library_items)






Library Catalog:
[Book] 'Python Crash Course' by Eric Matthes - Available
[Book] 'Clean Code' by Robert C. Martin - Available
[Magazine] 'National Geographic' (IssueNov 2025) - Available
You borrowed 'Python Crash Course'.
You borrowed 'National Geographic'.

Books borrowed by Nikhil:
  -Python Crash Course
  -National Geographic
Nikhil cannot borrow 'Python Crash Course' - already taken


AttributeError: 'BookItem' object has no attribute 'return_book'